In [2]:
import sys
sys.path.append("..")
from sklearn.model_selection import train_test_split
import numpy as np

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [3]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 14.910820722579956 to run!


## Load pickled multiview action data

In [4]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [5]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

['drink' 'play guitar' 'read' 'use vacuum cleaner' 'use laptop'
 'play guitar' 'call cellphone' 'play guitar' 'sit still' 'play guitar'
 'sit down' 'call cellphone' 'use vacuum cleaner' 'sit still' 'sit still'
 'drink' 'sit down' 'play game' 'lay down on sofa' 'walking'
 'write on paper' 'use laptop' 'use vacuum cleaner' 'drink'
 'write on paper' 'write on paper' 'sit still' 'cheer up' 'sit still'
 'lay down on sofa' 'play game' 'play guitar' 'sit down' 'sit down'
 'write on paper' 'call cellphone' 'stand up' 'use vacuum cleaner'
 'use laptop' 'use vacuum cleaner' 'play guitar' 'sit still' 'cheer up'
 'call cellphone' 'sit down' 'walking' 'drink' 'write on paper'
 'toss paper' 'cheer up' 'play guitar' 'toss paper' 'lay down on sofa'
 'walking' 'eat' 'call cellphone' 'use vacuum cleaner' 'sit still' 'eat'
 'play guitar' 'read' 'sit still' 'stand up' 'lay down on sofa' 'read'
 'lay down on sofa' 'eat' 'toss paper' 'eat' 'call cellphone' 'walking'
 'cheer up' 'stand up' 'toss paper' 'use 

/home/mobley/Documents/mines/machine-learning/CSCI470-Final/movingpose/preprocessing/moving_pose.py:103: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(X), labels


## Perform basic verification skeleton data is formatted correctly

In [6]:
for action_data in X:
    for i in range(0, len(action_data), 20):
        assert list(action_data[i][:-1]) == [0, 0, 0, 0, 0, 0, 0, 0, 0]
        assert action_data[i][-1] == i//20, f"{action_data[i][-1]} =/= {i//20}"

AssertionError: 2.0 =/= 0

## Create train/test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

## Train Action Classifier with normalized training data

In [ ]:
best_score = sys.maxsize
best_action_classifier = None
for n_neighbors in range(5, 15, 5):
    for n_training_neighbors in range(10, 40, 10):
        for alpha in np.arange(0.2, 0.8, 0.1):
            for beta in np.arange(0.1, 0.5, 0.1):
                for kappa in range(3, 10, 1):
                    nearest_descriptor_estimator = neighbors.NearestPoses(
                        n_neighbors=n_neighbors,
                        n_training_neighbors=n_training_neighbors,
                        alpha=alpha,
                        beta=beta,
                        kappa=kappa
                    )
                    for theta in np.arange(0.2, 0.8, 0.05):
                        for n in range(5, 15, 1):
                            action_classifier = classifiers.ActionClassifier(
                                nearest_descriptor_estimator=nearest_descriptor_estimator,
                                theta=theta,
                                n=n
                            )
                            action_classifier.fit(X_train, y_train)
                            y_pred = action_classifier.predict(X_test)

                            diff = 0
                            for pred, actual in zip(y_pred, y_test):
                                diff += abs(pred - actual)

                            if diff < best_score:
                                best_score = diff
                                best_action_classifier = action_classifier

## Print best action classifier and its score

In [ ]:
print(f"The best action classifier was {best_action_classifier}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
